In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.ablation import answer_question, answer_question_with_reasoning, answer_question_with_triplets, answer_question_reasoning_with_triplets
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
qd_df = pd.read_json('../../data/generated/musique-evaluation/question-decomposition.jsonl', orient='records', lines=True)
df = pd.merge(df.drop(columns=['question', 'question_decomposition']), qd_df, on='id', suffixes=('', ''))
# df = df.head(10)

print(df.shape)
df.head()

(100, 8)


,id,paragraphs,answer,answer_aliases,answerable,answers,question,question_decomposition
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar],Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Who is Mahmoud Mi..."
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",Berrien County,[Berrien County],True,[Berrien County],In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'To which city is ..."
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River],For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'On which river is..."
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith],Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Where was Tebesa ..."
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Snapper Foster,[Snapper Foster],True,[Snapper Foster],Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Who performed Nig..."


In [5]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [6]:
N_RUNS = 3
results = []

In [7]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "direct", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.6527385686209215,
  "fuzzy_match": 0.67
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6570573870573871,
  "fuzzy_match": 0.69
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.54,
  "f1": 0.6478814257637786,
  "fuzzy_match": 0.67
}


In [8]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_with_reasoning, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.51,
  "f1": 0.6258702212819859,
  "fuzzy_match": 0.61
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.55,
  "f1": 0.6750116354234003,
  "fuzzy_match": 0.69
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.53,
  "f1": 0.6733146657264304,
  "fuzzy_match": 0.67
}


In [9]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_with_triplets, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "triplets", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.61,
  "f1": 0.711921802054155,
  "fuzzy_match": 0.74
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.6837273576097105,
  "fuzzy_match": 0.7
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.6,
  "f1": 0.6956247338777048,
  "fuzzy_match": 0.72
}


In [10]:
for i in range(1, N_RUNS+1):
    _, scores = benchmark(df, answer_question_reasoning_with_triplets, perfect_retrieval_func, ignore_errors=True)
    results.append({**scores, "retrieval": "groundtruth", "context": "paragraphs", "qa": "reasoning+triplets", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.57,
  "f1": 0.6867976000607579,
  "fuzzy_match": 0.7
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.59,
  "f1": 0.7099034037857568,
  "fuzzy_match": 0.71
}


  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.56,
  "f1": 0.703116053852896,
  "fuzzy_match": 0.7
}


# Report

In [11]:
report_df = pd.DataFrame.from_records(results, columns=['context', 'retrieval', 'qa', 'run', 'exact_match', 'f1'])
report_df

,context,retrieval,qa,run,exact_match,f1
0,paragraphs,groundtruth,direct,1,0.56,0.652739
1,paragraphs,groundtruth,direct,2,0.55,0.657057
2,paragraphs,groundtruth,direct,3,0.54,0.647881
3,paragraphs,groundtruth,reasoning,1,0.51,0.625870
4,paragraphs,groundtruth,reasoning,2,0.55,0.675012
5,paragraphs,groundtruth,reasoning,3,0.53,0.673315
6,paragraphs,groundtruth,triplets,1,0.61,0.711922
7,paragraphs,groundtruth,triplets,2,0.59,0.683727
8,paragraphs,groundtruth,triplets,3,0.60,0.695625
9,paragraphs,groundtruth,reasoning+triplets,1,0.57,0.686798


In [12]:
report_df.drop(columns=['context', 'retrieval', 'run']).groupby(['qa']).mean()

,exact_match,f1
qa,,
direct,0.550000,0.652559
reasoning,0.530000,0.658066
reasoning+triplets,0.573333,0.699939
triplets,0.600000,0.697091
